## COD API, Matches --> explore, format, clean & reshape

#### import stuff to work with

In [1]:
import asyncio
import os
import sys
import dotenv
from pprint import pprint
import datetime
from datetime import datetime, timezone, timedelta
import pandas as pd
import numpy as np
import pickle

import callofduty
from callofduty import Mode, Platform, Title, TimeFrame, GameType

In [2]:
# Using SSO
# We're storing our SSO token in an .env file stored locally to separate our config from code (w. python-dotenv). An.env-template file (with help to retrieve token) is provided for you to edit and populate the variable(s)
# callofduty.py client .Login() goes through all the authentification steps and initiate a session to access protected routes
# The client is asynchronous thus the 'await style'
from dotenv import load_dotenv
load_dotenv()
client = await callofduty.Login(sso=os.environ["SSO"])

In [3]:
# we're storing our strings conversion module above current working (notebooks) directory
sys.path.insert(0, os.path.abspath('../wzkd'))
import labels
from labels import MODES_LABELS

In [4]:
# This time we're adding additional methods in the Call of Duty .py client only as there is no need to modify the HTTP class that already contains the endpoint we want to use
# Import the Class we want to modify

import urllib.parse
from typing import List, Optional, Union

from callofduty.client import Client
from callofduty.http import HTTP
from callofduty.http import Request


# following additional methods to be added in callofduty.client.py Client Class
# see notebooks/cod_api_doc.ipnyb for details

async def GetMatches(
    self, platform, username: str, title: Title, mode: Mode, **kwargs
):

    limit: int = kwargs.get("limit", 20)
    startTimestamp: int = kwargs.get("startTimestamp", 0)
    endTimestamp: int = kwargs.get("endTimestamp", 0)

    data: dict = (
        await self.http.GetPlayerMatches(
            platform,
            username,
            title.value,
            mode.value,
            limit,
            startTimestamp,
            endTimestamp,
        )
    )["data"] # API res was filtered out here

    return data


async def GetMatchesDetailed(
    self, platform, username: str, title: Title, mode: Mode, **kwargs
):

    limit: int = kwargs.get("limit", 20)
    startTimestamp: int = kwargs.get("startTimestamp", 0)
    endTimestamp: int = kwargs.get("endTimestamp", 0)

    return (
        await self.http.GetPlayerMatchesDetailed(
            platform,
            username,
            title.value,
            mode.value,
            limit,
            startTimestamp,
            endTimestamp,
        )
    )["data"]['matches'] # API res was filtered out here


async def GetMatchesSummary(
    self, platform, username: str, title: Title, mode: Mode, **kwargs
):

    limit: int = kwargs.get("limit", 20)
    startTimestamp: int = kwargs.get("startTimestamp", 0)
    endTimestamp: int = kwargs.get("endTimestamp", 0)

    return (
        await self.http.GetPlayerMatchesDetailed(
            platform,
            username,
            title.value,
            mode.value,
            limit,
            startTimestamp,
            endTimestamp,
        )
    )["data"]['summary'] # API res was filtered out here


# add our modified methods into callofduty Client Class

Client.GetMatches = GetMatches
Client.GetMatchesDetailed = GetMatchesDetailed
Client.GetMatchesSummary = GetMatchesSummary

# raw_matches_datailed has indeed two keys : summary and matches (the one that is filtered out in the Client)
matches = await client.GetMatchesDetailed("battle", "AMADEVS#1689", Title.ModernWarfare, Mode.Warzone, limit=20)


In [5]:
# option save your result if you want to work in offline mode
with open("matches.pkl", 'wb') as f:
    pickle.dump(matches, f)

### Overview of returned stats

In [6]:
# option : load previously saved offline data i
open_file = open("matches.pkl", "rb")
loaded_matches = pickle.load(open_file)
open_file.close()

In [7]:
pprint(loaded_matches[0], depth=2)

{'draw': False,
 'duration': 1651000,
 'gameType': 'wz',
 'map': 'mp_don4',
 'matchID': '6819668192065006886',
 'mode': 'br_brduos',
 'player': {'awards': {},
            'brMissionStats': {...},
            'clantag': 'lkf :',
            'loadout': [...],
            'loadouts': [...],
            'rank': 54.0,
            'team': 'team_forty_seven',
            'uno': '2621859779580650696',
            'username': 'gentil_renard'},
 'playerCount': 149,
 'playerStats': {'assists': 0.0,
                 'bonusXp': 0.0,
                 'challengeXp': 0.0,
                 'damageDone': 553.0,
                 'damageTaken': 383.0,
                 'deaths': 2.0,
                 'distanceTraveled': 353311.1,
                 'executions': 0.0,
                 'gulagDeaths': 1.0,
                 'gulagKills': 0.0,
                 'headshots': 0.0,
                 'kdRatio': 0.5,
                 'kills': 1.0,
                 'longestStreak': 0.0,
                 'matchXp': 12084.

#### raw data

In [8]:
raw = pd.DataFrame(loaded_matches)
raw.head(5)

,utcStartSeconds,utcEndSeconds,map,mode,matchID,duration,playlistName,version,gameType,playerCount,playerStats,player,teamCount,rankedTeams,draw,privateMatch
0,1636757782,1636759433,mp_don4,br_brduos,6819668192065006886,1651000,None,1,wz,149,"{'kills': 1.0, 'medalXp': 0.0, 'matchXp': 1208...","{'team': 'team_forty_seven', 'rank': 54.0, 'aw...",76,None,False,False
1,1636756390,1636758003,mp_don4,br_brduos,13033616674026392343,1613000,None,1,wz,150,"{'kills': 1.0, 'medalXp': 15.0, 'objectiveLast...","{'team': 'team_twelve', 'rank': 54.0, 'awards'...",76,None,False,False
2,1636755468,1636757083,mp_don4,br_brduos,14102694192743014733,1615000,None,1,wz,151,"{'kills': 2.0, 'medalXp': 40.0, 'objectiveTeam...","{'team': 'team_fifty_six', 'rank': 54.0, 'awar...",75,None,False,False
3,1636754316,1636755899,mp_don4,br_brduos,4313012347484911050,1583000,None,1,wz,149,"{'kills': 2.0, 'medalXp': 0.0, 'matchXp': 1005...","{'team': 'team_sixty', 'rank': 54.0, 'awards':...",76,None,False,False
4,1636753664,1636755235,mp_don4,br_brduos,17612358397501404898,1571000,None,1,wz,149,"{'kills': 1.0, 'medalXp': 30.0, 'matchXp': 498...","{'team': 'team_thirty_two', 'rank': 54.0, 'awa...",76,None,False,False


In [9]:
raw.keys()

Index(['utcStartSeconds', 'utcEndSeconds', 'map', 'mode', 'matchID',
       'duration', 'playlistName', 'version', 'gameType', 'playerCount',
       'playerStats', 'player', 'teamCount', 'rankedTeams', 'draw',
       'privateMatch'],
      dtype='object')

#### playerStats col

In [10]:
player_stats = raw['playerStats'].apply(pd.Series)
player_stats.head(20)

,kills,medalXp,objectiveTeamWiped,objectiveLastStandKill,matchXp,scoreXp,wallBangs,score,totalXp,headshots,...,objectiveBrKioskBuy,gulagDeaths,gulagKills,teamPlacement,objectiveMunitionsBoxTeammateUsed,objectiveMedalScoreSsKillPrecisionAirstrike,objectiveBrDownEnemyCircle3,objectiveBrDownEnemyCircle4,objectiveBrDownEnemyCircle2,objectiveReviver
0,7.0,345.0,5.0,3.0,3570.0,5342.0,0.0,3050.0,9257.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,20.0,NaN,1.0,9380.0,2625.0,0.0,1425.0,12025.0,0.0,...,1.0,2.0,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,30.0,1.0,NaN,2562.0,1050.0,0.0,850.0,3642.0,0.0,...,NaN,0.0,1.0,41.0,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,0.0,NaN,NaN,3992.0,800.0,0.0,300.0,4792.0,0.0,...,NaN,2.0,0.0,37.0,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,50.0,NaN,4.0,10629.0,5545.0,0.0,2725.0,16281.0,0.0,...,1.0,2.0,0.0,7.0,3.0,NaN,NaN,NaN,NaN,NaN
5,2.0,30.0,NaN,1.0,10448.0,3465.0,0.0,1725.0,13997.0,1.0,...,1.0,0.0,1.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN
6,12.0,670.0,9.0,1.0,4325.0,7832.0,0.0,4625.0,12827.0,5.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
7,0.0,100.0,NaN,NaN,0.0,400.0,0.0,0.0,500.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,5.0,390.0,3.0,2.0,2650.0,7026.0,0.0,2025.0,10066.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.0,50.0,NaN,NaN,0.0,200.0,0.0,0.0,250.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
player_stats.keys()

Index(['kills', 'medalXp', 'objectiveTeamWiped', 'objectiveLastStandKill',
       'matchXp', 'scoreXp', 'wallBangs', 'score', 'totalXp', 'headshots',
       'assists', 'challengeXp', 'rank', 'scorePerMinute', 'distanceTraveled',
       'deaths', 'objectiveDestroyedEquipment', 'kdRatio', 'bonusXp',
       'timePlayed', 'executions', 'nearmisses', 'objectiveBrCacheOpen',
       'percentTimeMoving', 'miscXp', 'longestStreak', 'damageDone',
       'damageTaken', 'teamSurvivalTime', 'objectiveBrDownEnemyCircle1',
       'objectiveBrMissionPickupTablet', 'objectiveBrKioskBuy', 'gulagDeaths',
       'gulagKills', 'teamPlacement', 'objectiveMunitionsBoxTeammateUsed',
       'objectiveMedalScoreSsKillPrecisionAirstrike',
       'objectiveBrDownEnemyCircle3', 'objectiveBrDownEnemyCircle4',
       'objectiveBrDownEnemyCircle2', 'objectiveReviver'],
      dtype='object')

In [12]:
player_stats[['headshots', 'distanceTraveled', 'teamSurvivalTime', 'objectiveBrKioskBuy']].head(5)

,headshots,distanceTraveled,teamSurvivalTime,objectiveBrKioskBuy
0,1.0,217923.86,NaN,NaN
1,0.0,257729.83,1067712.0,1.0
2,0.0,219496.16,323280.0,NaN
3,0.0,204363.31,466896.0,NaN
4,0.0,345855.84,1175664.0,1.0


#### player col

In [13]:
player = raw['player'].apply(pd.Series)
player.head(5)

,team,rank,awards,username,uno,clantag,loadouts,brMissionStats,loadout
0,team_twenty_three,54.0,{},gentil_renard,2621859779580650696,lkf :,[{'primaryWeapon': {'name': 'iw8_ar_t9slowhand...,"{'missionsComplete': 0, 'totalMissionXpEarned'...",[{'primaryWeapon': {'name': 'iw8_ar_t9slowhand...
1,team_twenty,54.0,{},gentil_renard,2621859779580650696,lkf :,"[{'primaryWeapon': {'name': 'iw8_sh_aalpha12',...","{'missionsComplete': 0, 'totalMissionXpEarned'...","[{'primaryWeapon': {'name': 'iw8_sh_aalpha12',..."
2,team_nineteen,54.0,{},gentil_renard,2621859779580650696,lkf :,"[{'primaryWeapon': {'name': 'iw8_sh_aalpha12',...","{'missionsComplete': 0, 'totalMissionXpEarned'...","[{'primaryWeapon': {'name': 'iw8_sh_aalpha12',..."
3,team_eight,54.0,{},gentil_renard,2621859779580650696,lkf :,"[{'primaryWeapon': {'name': 'iw8_sh_aalpha12',...","{'missionsComplete': 0, 'totalMissionXpEarned'...","[{'primaryWeapon': {'name': 'iw8_sh_aalpha12',..."
4,team_thirty_one,54.0,{},gentil_renard,2621859779580650696,lkf :,"[{'primaryWeapon': {'name': 'iw8_sh_aalpha12',...","{'missionsComplete': 0, 'totalMissionXpEarned'...","[{'primaryWeapon': {'name': 'iw8_sh_aalpha12',..."


In [14]:
player.keys()

Index(['team', 'rank', 'awards', 'username', 'uno', 'clantag', 'loadouts',
       'brMissionStats', 'loadout'],
      dtype='object')

#### Deep dive into player.loadout entry

In [18]:
# Each entry of 'loadout' is a list of dict. Either one dict (if 1 loadout) or more(2 or more loadouts taken)
player['loadout'][11][0]

{'primaryWeapon': {'name': 'iw8_ar_falima',
  'label': None,
  'imageLoot': None,
  'imageIcon': None,
  'variant': '0',
  'attachments': [{'name': 'fmj',
    'label': None,
    'image': None,
    'category': None},
   {'name': 'reflexmini2', 'label': None, 'image': None, 'category': None},
   {'name': 'xmags', 'label': None, 'image': None, 'category': None},
   {'name': 'silencer3', 'label': None, 'image': None, 'category': None},
   {'name': 'laserrange', 'label': None, 'image': None, 'category': None}]},
 'secondaryWeapon': {'name': 'iw8_sn_t9accurate',
  'label': None,
  'imageLoot': None,
  'imageIcon': None,
  'variant': '0',
  'attachments': [{'name': 'stockcust',
    'label': None,
    'image': None,
    'category': None},
   {'name': 'laserrange', 'label': None, 'image': None, 'category': None},
   {'name': 'silencer2', 'label': None, 'image': None, 'category': None},
   {'name': 'barcust', 'label': None, 'image': None, 'category': None},
   {'name': 'pistolgrip06', 'label': N

In [ ]:
# second loadout structure 
player['loadout'][11][1].keys()

In [21]:
player['loadout'][11][0]['primaryWeapon'].keys()

dict_keys(['name', 'label', 'imageLoot', 'imageIcon', 'variant', 'attachments'])

#### expanded data : raw + playerStats & player columns expanded + player.loadout expanded

In [22]:
expanded = pd.DataFrame(loaded_matches)
expanded = pd.concat([expanded.drop(['playerStats'], axis=1), expanded['playerStats'].apply(pd.Series)], axis=1)
expanded = pd.concat([expanded.drop(['player'], axis=1), expanded['player'].apply(pd.Series)], axis=1)
expanded = expanded.drop(['brMissionStats'], axis = 1)

# player.loadout column
expanded = pd.concat([expanded.drop(['loadout'], axis=1), expanded['loadout'].apply(pd.Series)], axis=1)
for col in range(0,3):
    if col in expanded.columns:
        expanded[col] = expanded[col].apply(lambda x: [x['primaryWeapon']['name'],x['secondaryWeapon']['name']] if not str(x) == 'nan' else np.nan)
        expanded = expanded.rename(columns={col: f"loadout_{str(col +1)}"})
expanded.head(15)

,utcStartSeconds,utcEndSeconds,map,mode,matchID,duration,playlistName,version,gameType,playerCount,...,awards,username,uno,clantag,loadouts,loadout_1,loadout_2,loadout_3,3,4
0,1636580856,1636581728,mp_don4,br_dmz_plnbld,3730567781417063940,872000,None,1,wz,100,...,{},gentil_renard,2621859779580650696,lkf :,[{'primaryWeapon': {'name': 'iw8_ar_t9slowhand...,"[iw8_ar_t9slowhandling, iw8_ar_t9damage]","[iw8_ar_t9british, iw8_ar_mike4]","[iw8_ar_t9standard, iw8_sm_t9fastfire]",{'primaryWeapon': {'name': 'iw8_sm_t9standard'...,"{'primaryWeapon': {'name': 'iw8_sm_t9spray', '..."
1,1636402967,1636404602,mp_don4,br_brtrios,7308668688828180189,1635000,None,1,wz,150,...,{},gentil_renard,2621859779580650696,lkf :,"[{'primaryWeapon': {'name': 'iw8_sh_aalpha12',...","[iw8_sh_aalpha12, iw8_sn_t9powersemi]","[iw8_pi_t9semiauto, iw8_fists]","[iw8_pi_t9semiauto, iw8_fists]",NaN,NaN
2,1636402485,1636404084,mp_don4,br_brtrios,2261680546254418765,1599000,None,1,wz,150,...,{},gentil_renard,2621859779580650696,lkf :,"[{'primaryWeapon': {'name': 'iw8_sh_aalpha12',...","[iw8_sh_aalpha12, iw8_sn_t9powersemi]","[iw8_pi_t9semiauto, iw8_fists]",NaN,NaN,NaN
3,1636401873,1636403494,mp_don4,br_brtrios,2897643815664392564,1621000,None,1,wz,150,...,{},gentil_renard,2621859779580650696,lkf :,"[{'primaryWeapon': {'name': 'iw8_sh_aalpha12',...","[iw8_sh_aalpha12, iw8_sn_t9powersemi]","[iw8_pi_t9semiauto, iw8_fists]","[iw8_ar_t9season6, iw8_fists]",NaN,NaN
4,1636400510,1636402048,mp_don4,br_brquads,3761077364910334296,1538000,None,1,wz,160,...,{},gentil_renard,2621859779580650696,lkf :,"[{'primaryWeapon': {'name': 'iw8_sh_aalpha12',...","[iw8_sh_aalpha12, iw8_sn_t9powersemi]","[iw8_pi_t9semiauto, iw8_fists]","[iw8_ar_t9season6, iw8_fists]",NaN,NaN
5,1636399097,1636400756,mp_don4,br_brtrios,2976279808342734071,1659000,None,1,wz,150,...,{},gentil_renard,2621859779580650696,lkf :,"[{'primaryWeapon': {'name': 'iw8_sh_dpapa12', ...","[iw8_sh_dpapa12, iw8_sn_t9powersemi]","[iw8_pi_t9semiauto, iw8_fists]",NaN,NaN,NaN
6,1636381287,1636382341,mp_don4,br_dmz_plnbld,14323063713814370440,1054000,None,1,wz,100,...,{},gentil_renard,2621859779580650696,lkf :,"[{'primaryWeapon': {'name': 'iw8_sh_dpapa12', ...","[iw8_sh_dpapa12, iw8_sn_t9powersemi]","[iw8_ar_valpha, iw8_sn_t9cannon]","[iw8_ar_falima, iw8_sn_t9accurate]","{'primaryWeapon': {'name': 'iw8_ar_valpha', 'l...",NaN
7,1636222919,1636223792,mp_don4,br_dmz_plnbld,2496317311051128063,873000,None,1,wz,105,...,{},gentil_renard,2621859779580650696,lkf :,"[{'primaryWeapon': {'name': 'iw8_sh_dpapa12', ...","[iw8_sh_dpapa12, iw8_sn_t9powersemi]",NaN,NaN,NaN,NaN
8,1636222133,1636222782,mp_don4,br_dmz_plnbld,10843599778053414364,649000,None,1,wz,100,...,{},gentil_renard,2621859779580650696,lkf :,"[{'primaryWeapon': {'name': 'iw8_sh_dpapa12', ...","[iw8_sh_dpapa12, iw8_sn_t9powersemi]",NaN,NaN,NaN,NaN
9,1636135569,1636136379,mp_don4,br_dmz_plnbld,7915247606251655745,810000,None,1,wz,106,...,{},gentil_renard,2621859779580650696,lkf :,"[{'primaryWeapon': {'name': 'iw8_sh_dpapa12', ...","[iw8_sh_dpapa12, iw8_sn_t9powersemi]",NaN,NaN,NaN,NaN


#### Functions

In [45]:
def getLastMatchId(matches):
    """ Extract last (Battle Royale) Match ID from list of last matches """
    list_br_match = [match for match in matches if "br_br" in match["mode"]]
    last_match_id = int(list_br_match[0]["matchID"]) if len(list_br_match) > 0 else None
    return last_match_id


def getGamertag(matches):
    """ 
    A player can be searchable with a given name but having a different gamertag in-game
    We need that gamertag if we were to analyse the "one match stats" endpoint (as done in match_format.py)
    from player's perspective
    """
    return matches[0]['player']['username']

def MatchesToDf(matches):
    """
    Convert Matches API result to a DataFrame we we can perform our aggregations nicely, later.
    Expand some entries (i.e player, playerstats) that are deeply nested.
    Filter out / retains columns.
    
    Parameters
    ----------
    matches : result from COD API "matches" endpoint ; FYI formated as : 
        list[
                dict{ match 1 stats },
                dict{ match 2 stats },
                dict{ 20 matches max },
            ]
    
    Returns
    -------
    DataFrame, (max 20) matches as rows, matches' stats as columns/values
    """    
    
    
    keep_cols =  [
        'mode',
        'utcStartSeconds',
        'utcEndSeconds',
        'timePlayed',
        'teamPlacement',
        'kdRatio', 
        'kills', 
        'deaths', 
        'assists', 
        'damageDone',
        'damageTaken',
        'gulagKills',
        'percentTimeMoving',
        'duration',
        ]

    df = pd.DataFrame(matches)
    
    # column playerStats is a series of dict, we can expand it easily and append, then drop the original
    df = pd.concat([df.drop(['playerStats'], axis=1), df['playerStats'].apply(pd.Series)], axis=1)
    
    # colum player has more depth
    # once expanded, it has a column 'loadout' : a series of list of dict (either one or more, we will keep 1 to max 3)
    # and also brMissionStats (mostly empty ?, a col only present in BR matches) that we aren't interested in
    
    df = pd.concat([df.drop(['player'], axis=1), df['player'].apply(pd.Series)], axis=1)
    if 'brMissionStats' in df.columns:
        df = df.drop(['brMissionStats'], axis = 1)
    df = pd.concat([df.drop(['loadout'], axis=1), df['loadout'].apply(pd.Series)], axis=1)
    for col in range(0,3):
        if col in df.columns:
            df[col] = df[col].apply(lambda x: f"{x['primaryWeapon']['name']} - {x['secondaryWeapon']['name']}" if not str(x) == 'nan' else np.nan)
            col_name = "loadout_" + str(col +1)
            df = df.rename(columns={col: f"loadout_{str(col +1)}"})   
            keep_cols.append(col_name)
    
    return df[keep_cols]

In [46]:
df = MatchesToDf(loaded_matches)
df.head(3)

,mode,utcStartSeconds,utcEndSeconds,timePlayed,teamPlacement,kdRatio,kills,deaths,assists,damageDone,damageTaken,gulagKills,percentTimeMoving,duration,loadout_1,loadout_2,loadout_3
0,br_dmz_plnbld,1636580856,1636581728,966.0,NaN,0.7,7.0,10.0,2.0,2996.0,1237.0,NaN,83.821800,872000,iw8_ar_t9slowhandling - iw8_ar_t9damage,iw8_ar_t9british - iw8_ar_mike4,iw8_ar_t9standard - iw8_sm_t9fastfire
1,br_brtrios,1636402967,1636404602,1165.0,14.0,0.5,2.0,4.0,0.0,688.0,770.0,1.0,54.621853,1635000,iw8_sh_aalpha12 - iw8_sn_t9powersemi,iw8_pi_t9semiauto - iw8_fists,iw8_pi_t9semiauto - iw8_fists
2,br_brtrios,1636402485,1636404084,406.0,41.0,1.5,3.0,2.0,0.0,875.0,267.0,1.0,94.594600,1599000,iw8_sh_aalpha12 - iw8_sn_t9powersemi,iw8_pi_t9semiauto - iw8_fists,NaN


In [25]:
last_match_id = getLastMatchId(loaded_matches)
last_match_id

7308668688828180189

In [26]:
def MatchesStandardize(df):
    """
    A first layer of standadization (as : properly formated) to our matches DataFrame
    For further aggregations / better readibility of our data
    
    Returns
    -------
    DataFrame : matches as rows, cleaned matches/player stats as columns
    """
    
    int_cols =  [
        'teamPlacement', 
        'kills', 
        'deaths', 
        'assists', 
        'gulagKills', 
        'damageDone',
        'damageTaken'
        ]
    
    float_cols = [
        'kdRatio',
        'percentTimeMoving'
        ]
    
    ts_cols = [
        'utcStartSeconds',
        'utcEndSeconds'
        ]
    
    columns_labels = {
        'utcEndSeconds':'Ended at',
        'utcStartSeconds':'Started at',
        'timePlayed': 'Playtime',
        'teamPlacement':'#',
        'kdRatio':'KD',
        'damageDone':'Damage >',
        'damageTaken':'Damage <',
        'gulagKills':'Gulag',
        'headshots':'% headshots',
        'percentTimeMoving':'% moving',
        'duration':'Game duration'
        }

    df = df.fillna(0) # sometimes (Plunder only ?) rank, gulag is NaN
    df[int_cols] = df[int_cols].astype(int)
    df[float_cols] = df[float_cols].astype(float).round(1) # still renders 0.0000 in streamlit but ugly hacks exists
    
    # specials
    df['utcEndSeconds'] = df['utcEndSeconds'].apply(lambda x: datetime.fromtimestamp(x))
    df['utcStartSeconds'] = df['utcStartSeconds'].apply(lambda x: datetime.fromtimestamp(x))
    
    df['duration'] = df['duration'].apply(lambda x: x/1000).apply(lambda x: pd.to_datetime(x, unit='s').strftime('%M')) # API duration is in seconds x1000
    df['timePlayed'] = df['timePlayed'].apply(lambda x: pd.to_datetime(x, unit='s').strftime('%M:%S')) # API timePlayed is in seconds
    df['gulagKills'] = df['gulagKills'].map({1:'W', 0:'L'})
    for col in ['loadout_1', 'loadout_2', 'loadout_3']:
            df.fillna({col:'-'}, inplace=True) if col in df.columns else None
    
    df = df.replace({"mode": MODES_LABELS})
    df = df.rename(columns=columns_labels)
    df.columns = df.columns.str.capitalize()
    df = df.rename({"Kd":"KD"}, axis=1)
     
    return df

In [27]:
raw = pd.DataFrame(loaded_matches)
df = MatchesToDf(raw)
df = MatchesStandardize(df)
df.head(6)

,Mode,Started at,Ended at,Playtime,#,KD,Kills,Deaths,Assists,Damage >,Damage <,Gulag,% moving,Game duration,Loadout_1,Loadout_2,Loadout_3
0,Plunder,2021-11-10 22:47:36,2021-11-10 23:02:08,16:06,0,0.7,7,10,2,2996,1237,L,83.8,14,iw8_ar_t9slowhandling - iw8_ar_t9damage,iw8_ar_t9british - iw8_ar_mike4,iw8_ar_t9standard - iw8_sm_t9fastfire
1,Trios,2021-11-08 21:22:47,2021-11-08 21:50:02,19:25,14,0.5,2,4,0,688,770,W,54.6,27,iw8_sh_aalpha12 - iw8_sn_t9powersemi,iw8_pi_t9semiauto - iw8_fists,iw8_pi_t9semiauto - iw8_fists
2,Trios,2021-11-08 21:14:45,2021-11-08 21:41:24,06:46,41,1.5,3,2,0,875,267,W,94.6,26,iw8_sh_aalpha12 - iw8_sn_t9powersemi,iw8_pi_t9semiauto - iw8_fists,0
3,Trios,2021-11-08 21:04:33,2021-11-08 21:31:34,09:16,37,0.0,0,3,2,288,400,L,45.1,27,iw8_sh_aalpha12 - iw8_sn_t9powersemi,iw8_pi_t9semiauto - iw8_fists,iw8_ar_t9season6 - iw8_fists
4,Quads,2021-11-08 20:41:50,2021-11-08 21:07:28,20:56,7,1.3,4,3,0,1120,414,L,62.7,25,iw8_sh_aalpha12 - iw8_sn_t9powersemi,iw8_pi_t9semiauto - iw8_fists,iw8_ar_t9season6 - iw8_fists
5,Trios,2021-11-08 20:18:17,2021-11-08 20:45:56,21:29,12,1.0,2,2,0,343,469,W,92.5,27,iw8_sh_dpapa12 - iw8_sn_t9powersemi,iw8_pi_t9semiauto - iw8_fists,0


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Mode           20 non-null     object        
 1   Started at     20 non-null     datetime64[ns]
 2   Ended at       20 non-null     datetime64[ns]
 3   Playtime       20 non-null     object        
 4   #              20 non-null     int64         
 5   KD             20 non-null     float64       
 6   Kills          20 non-null     int64         
 7   Deaths         20 non-null     int64         
 8   Assists        20 non-null     int64         
 9   Damage >       20 non-null     int64         
 10  Damage <       20 non-null     int64         
 11  Gulag          20 non-null     object        
 12  % moving       20 non-null     float64       
 13  Game duration  20 non-null     object        
 14  Loadout_1      20 non-null     object        
 15  Loadout_2      20 non-nul

In [29]:
loadout_cols = df.columns[df.columns.str.startswith('Loadout')].tolist()
loadout_cols

['Loadout_1', 'Loadout_2', 'Loadout_3']

In [30]:
def MatchesPerDay(df):
    """
    Final layer applied to our list of matches with stats, to render them in our Streamlit App
    Streamlit/basic AgGrid does not render well (aka w. blank rows etc.) multi indexed df
    So we structure and display our data differently (a dictionary instead of a df), in a daily manner
    
    Returns
    -------
    Dictionary :
    {
        "str_weekday_1":df-of-matches-that-day,
        "str_weekday_2":df-of-matches-that-day,
        (...)
    }
    """
    
    drop_cols =  [
        'Started at',
        'Playtime',
        '% moving',
        'Game duration'
        ]
    
    keep_cols = [
        'End time',
        'Mode',
        '#',
        'KD',
        'Kills',
        'Deaths',
        'Assists',
        'Damage >',
        'Damage <',
        'Gulag'
    ]
    
    loadout_cols = df.columns[df.columns.str.startswith('Loadout')].tolist()
    df[loadout_cols] = df[loadout_cols].replace(0, '-') # else can't concat Loadouts cols
    
    # 1. --- initial formating : datetime, concat loadouts cols ---
    
    df = df.drop(drop_cols, axis = 1)
    df['End time'] = df['Ended at'].dt.time
    
    def concat_loadouts(df, columns):
        return pd.Series(map(' , '.join, df[columns].values.tolist()),index = df.index)
    
    df['Weapons'] = concat_loadouts(df, loadout_cols)
    keep_cols = [*keep_cols, *['Weapons']]
    
    # 2. --- build result {"day1": df-matches-that-day, "day2": df...} ---
    
    list_df = [g for n, g in df.groupby(pd.Grouper(key='Ended at',freq='D'))]
    list_df = [df for df in list_df if not df.empty]
    list_days = [df['Ended at'].tolist()[0].strftime('%A') for df in list_df]
    
    # make sure we display latest day first, then build dictionary
    for list_ in [list_days, list_df]:
        list_.reverse()
    day_matches = dict(zip(list_days, list_df))
    
    # 3. --- some more (re)formating ---
    
    # for some reason (me ? ^-^, Grouper => Series?) couldnt' modify df before building the result, must iterate again 
    for k, v in day_matches.items():
        day_matches[k] = day_matches[k][keep_cols]
      
    return day_matches


def AggStats(df):
    """
    We want to add aggregated stats for each day/df of matches we got from MatchesPerDay()
    Each daily aggregation will be rendered in our Streamlit App on top of each list of matches
    
    Returns
    -------
    Dictionary :
    {
    'Kills':total n kills that day,
    'Deaths': total deaths,
    'KD': kills/deaths,
    'Gulags': win % ,
    'Played': count matches -
    }
    """

    agg_func = {
        "Mode":"count",
        "Kills":"sum",
        "Deaths":"sum"
    }
    
    kd = (df.Kills.sum() / df.Deaths.sum()).round(2)
    gulagWinRatio = int((df.Gulag.str.count("W").sum() *100) / len(df))
    
    dict_ = df.agg(agg_func).to_dict()
    dict_.update({'KD': kd})
    dict_.update({'Gulags': gulagWinRatio})
    dict_['Played'] = dict_.pop('Mode')
    
    
    return dict_

In [31]:
raw = pd.DataFrame(loaded_matches)
df = MatchesToDf(raw)
df = MatchesStandardize(df)
daily = MatchesPerDay(df)

In [34]:
for day in daily.keys():
    print(day)
    day_stats = AggStats(daily[day])
    print(day_stats)
    display(daily[day])
    #print(daily[day]['Mode'].values)

Wednesday
{'Kills': 6, 'Deaths': 7, 'KD': 0.86, 'Gulags': 25, 'Played': 4}


,End time,Mode,#,KD,Kills,Deaths,Assists,Damage >,Damage <,Gulag,Weapons
15,21:39:49,Quads,23,2.0,4,2,0,1026,374,W,"iw8_ar_valpha - iw8_sn_t9cannon , iw8_pi_t9sem..."
14,21:55:34,Quads,7,2.0,2,1,4,1145,100,L,"iw8_ar_falima - iw8_sn_t9accurate , - , -"
13,22:21:50,Quads,11,0.0,0,2,0,266,475,L,"iw8_ar_valpha - iw8_sn_t9cannon , iw8_pi_t9sem..."
12,22:46:05,Quads,30,0.0,0,2,0,26,326,L,"iw8_ar_valpha - iw8_sn_t9cannon , iw8_pi_t9sem..."


Monday
{'Kills': 23, 'Deaths': 26, 'KD': 0.88, 'Gulags': 50, 'Played': 6}


,End time,Mode,#,KD,Kills,Deaths,Assists,Damage >,Damage <,Gulag,Weapons
6,15:39:01,Plunder,0,1.0,12,12,1,3581,1391,L,"iw8_sh_dpapa12 - iw8_sn_t9powersemi , iw8_ar_v..."
5,20:45:56,Trios,12,1.0,2,2,0,343,469,W,"iw8_sh_dpapa12 - iw8_sn_t9powersemi , iw8_pi_t..."
4,21:07:28,Quads,7,1.3,4,3,0,1120,414,L,"iw8_sh_aalpha12 - iw8_sn_t9powersemi , iw8_pi_..."
3,21:31:34,Trios,37,0.0,0,3,2,288,400,L,"iw8_sh_aalpha12 - iw8_sn_t9powersemi , iw8_pi_..."
2,21:41:24,Trios,41,1.5,3,2,0,875,267,W,"iw8_sh_aalpha12 - iw8_sn_t9powersemi , iw8_pi_..."
1,21:50:02,Trios,14,0.5,2,4,0,688,770,W,"iw8_sh_aalpha12 - iw8_sn_t9powersemi , iw8_pi_..."


Saturday
{'Kills': 5, 'Deaths': 10, 'KD': 0.5, 'Gulags': 0, 'Played': 2}


...

,End time,Mode,#,KD,Kills,Deaths,Assists,Damage >,Damage <,Gulag,Weapons
10,19:03:17,Plunder,0,1.0,12,12,3,2958,1256,L,"iw8_sh_dpapa12 - iw8_sn_t9powersemi , - , -"
9,19:19:39,Plunder,0,0.0,0,0,0,0,0,L,"iw8_sh_dpapa12 - iw8_sn_t9powersemi , - , -"


Thursday
{'Kills': 12, 'Deaths': 8, 'KD': 1.5, 'Gulags': 0, 'Played': 1}


,End time,Mode,#,KD,Kills,Deaths,Assists,Damage >,Damage <,Gulag,Weapons
11,19:17:52,Rumble,2,1.5,12,8,4,2925,823,L,"iw8_ar_falima - iw8_sn_t9accurate , - , -"


Tuesday
{'Kills': 5, 'Deaths': 8, 'KD': 0.62, 'Gulags': 25, 'Played': 4}


,End time,Mode,#,KD,Kills,Deaths,Assists,Damage >,Damage <,Gulag,Weapons
19,22:51:37,Quads,22,0.7,2,3,1,817,655,W,"iw8_ar_falima - iw8_sn_t9accurate , iw8_pi_t9s..."
18,23:07:45,Quads,26,0.0,0,2,0,66,456,L,"iw8_ar_valpha - iw8_sn_t9cannon , iw8_pi_t9sem..."
17,23:19:33,Quads,31,0.0,0,2,0,2,336,L,"iw8_ar_valpha - iw8_sn_t9cannon , iw8_pi_t9sem..."
16,23:28:27,Quads,10,3.0,3,1,3,883,440,L,"iw8_ar_falima - iw8_sn_t9accurate , - , -"


In [33]:
for k, v in daily.items():
    print(k)
    print(AggStats(v))

Wednesday
{'Kills': 6, 'Deaths': 7, 'KD': 0.86, 'Gulags': 25, 'Played': 4}
Monday
{'Kills': 23, 'Deaths': 26, 'KD': 0.88, 'Gulags': 50, 'Played': 6}
Saturday
{'Kills': 5, 'Deaths': 10, 'KD': 0.5, 'Gulags': 0, 'Played': 2}
Friday
{'Kills': 12, 'Deaths': 12, 'KD': 1.0, 'Gulags': 0, 'Played': 2}
Thursday
{'Kills': 12, 'Deaths': 8, 'KD': 1.5, 'Gulags': 0, 'Played': 1}
Tuesday
{'Kills': 5, 'Deaths': 8, 'KD': 0.62, 'Gulags': 25, 'Played': 4}


In [ ]:
df = daily["Thursday"].copy(deep=True)
df